In [1]:
import libsbml
from utils import functions
from utils import graph_functions
from utils import complete_tests
import random
from collections import defaultdict
from copy import deepcopy
import os
import pickle
import numpy as np

In [2]:
description = {
    'species': [],
    'reactions': [],
    'connections': [],
    'altered': ''
}

In [3]:
def alter_pathway(input_filename, output_filename, alterations):
    reader = libsbml.SBMLReader()
    document = reader.readSBML(input_filename)
    model = document.getModel()

    for i in range(model.getNumParameters()):
        param = model.getParameter(i)
        if param.getId() == alterations:
            previous_value = param.getValue()
            param.setValue(previous_value * 100)
            modified_sbml = libsbml.writeSBML(document, output_filename)
            return 

In [4]:
def generate_test_set(filename):
    reader = libsbml.SBMLReader()
    document = reader.readSBML(filename)
    model = document.getModel()
    
    for i in range(model.getNumParameters()):
        base_dir = 'altered_pathways2'
        alteration_dir = os.path.join(base_dir, 'reaction_' + str(i))
        file_dir = os.path.join(alteration_dir, 'reaction_' + str(i) + '100x.xml')
        os.makedirs(alteration_dir, exist_ok=True)
        
        param = model.getParameter(i)

        alter_pathway(filename, file_dir, param.getId())

In [5]:
generate_test_set('BIOMD0000000026.xml')

In [6]:
def convert_sbml_to_network(file_path):
    
    reader = libsbml.SBMLReader()
    document = reader.readSBML(file_path)
    if document.getNumErrors() > 0:
        print("Errors occurred while loading the SBML file.")
    model = document.getModel()
    
    num_reactions = model.getNumReactions()
    num_species = model.getNumSpecies()
    
    print('number reactions', model.getNumReactions())
    print('number species', model.getNumSpecies()) 
    
    for i in range(num_species):
        description['species'].append(model.getSpecies(i).getId())
    
    for i in range(num_reactions):
        description['reactions'].append([model.getReaction(i).getId(), 0, 1])
        num_reactants = model.getReaction(i).getNumReactants()
        num_products = model.getReaction(i).getNumProducts()
        formula = model.getReaction(i).getKineticLaw().getFormula()
        formula = formula.replace('(','')
        formula = formula.replace(' ', '')
        formula_splitted = formula.split('-')
        
        if len(formula_splitted) >= 2:
            description['reactions'].append([str(model.getReaction(i).getId() + 'revert'), 0, 1])
        
        for j in range(num_reactants):
            reactant = model.getReaction(i).getReactant(j).getSpecies()
            description['connections'].append([reactant, model.getReaction(i).getId()])
            if len(formula_splitted) == 2:
                description['connections'].append([str(model.getReaction(i).getId() + 'revert'), reactant])
            
        for k in range(num_products):
            product = model.getReaction(i).getProduct(k).getSpecies()
            description['connections'].append([model.getReaction(i).getId(), product])
            if len(formula_splitted) == 2:
                description['connections'].append([product, str(model.getReaction(i).getId() + 'revert')])

In [7]:
convert_sbml_to_network('BIOMD0000000026.xml')

number reactions 10
number species 11


In [8]:
for i in range(len(description['reactions'])):
    desc = deepcopy(description)
    desc['altered'] = desc['reactions'][i][0]
    base_dir = 'altered_pathways2'
    alteration_dir = os.path.join(base_dir, 'reaction_' + str(i))
    description_path = os.path.join(alteration_dir, 'description')
    description_file = open(description_path, 'wb')
    pickle.dump(desc, description_file)
    description_file.close()
    

In [9]:
desc_path = 'altered_pathways2/reaction_4/description'
desc_file = open(desc_path, 'rb')
temp = pickle.load(desc_file)
desc_file.close()
temp

{'species': ['M',
  'Mp',
  'Mpp',
  'MAPKK',
  'MKP3',
  'M_MAPKK',
  'Mp_MAPKK',
  'Mpp_MKP3',
  'Mp_MKP3_dep',
  'Mp_MKP3',
  'M_MKP3'],
 'reactions': [['v1a', 0, 1],
  ['v1arevert', 0, 1],
  ['v1b', 0, 1],
  ['v2a', 0, 1],
  ['v2arevert', 0, 1],
  ['v2b', 0, 1],
  ['v3a', 0, 1],
  ['v3arevert', 0, 1],
  ['v3b', 0, 1],
  ['v3c', 0, 1],
  ['v3crevert', 0, 1],
  ['v4a', 0, 1],
  ['v4arevert', 0, 1],
  ['v4b', 0, 1],
  ['v4c', 0, 1],
  ['v4crevert', 0, 1]],
 'connections': [['M', 'v1a'],
  ['v1arevert', 'M'],
  ['MAPKK', 'v1a'],
  ['v1arevert', 'MAPKK'],
  ['v1a', 'M_MAPKK'],
  ['M_MAPKK', 'v1arevert'],
  ['M_MAPKK', 'v1b'],
  ['v1b', 'Mp'],
  ['v1b', 'MAPKK'],
  ['Mp', 'v2a'],
  ['v2arevert', 'Mp'],
  ['MAPKK', 'v2a'],
  ['v2arevert', 'MAPKK'],
  ['v2a', 'Mp_MAPKK'],
  ['Mp_MAPKK', 'v2arevert'],
  ['Mp_MAPKK', 'v2b'],
  ['v2b', 'Mpp'],
  ['v2b', 'MAPKK'],
  ['Mpp', 'v3a'],
  ['v3arevert', 'Mpp'],
  ['MKP3', 'v3a'],
  ['v3arevert', 'MKP3'],
  ['v3a', 'Mpp_MKP3'],
  ['Mpp_MKP3', 'v3arev

In [2]:
total_species = 0
uncertain_species = 0
correct_species = 0
true_positives = 0
false_positives = 0
false_negatives = 0
true_negatives = 0 
deltas_normal = []
deltas_uncertain = []

complete_results = defaultdict()
for directory in list(os.listdir('altered_pathways2')):
    if directory == '.DS_Store':
        continue
    dir = os.path.join('altered_pathways2', directory)
    files = os.listdir(dir)
    description_index = 0
    altered_file_index = 0
    
    for i in range(len(files)):
        if files[i] == 'description':
            description_index = i
        elif files[i] != 'description' and files[i] != 'results':
            altered_file_index = i
    
            
    res = complete_tests.complete_tests2(
            'BIOMD0000000026.xml',
            os.path.join(dir, files[altered_file_index]),
            os.path.join(dir, 'description'),
            []
        )
    
    '''complete_results_path = os.path.join(dir, 'results')
    complete_results_file = open(complete_results_path, 'wb')
    pickle.dump(res, complete_results_file)
    complete_results_file.close()'''
        
    insights, all, uncertain, d_normal, d_uncertain = complete_tests.get_insights_confidence_intervals('altered', res, 1, detailed_classification=False, threshold=0, ignore_uncertain=False)
    
    total_species += all
    uncertain_species += uncertain
    
    for d in d_normal:
        deltas_normal.append(d)
    for d in d_uncertain:
        deltas_uncertain.append(d)
        
    total_count = 0
    correct_count = 0
    for specie in insights.keys():
        
        if insights[specie][0] == 0 or insights[specie][1][1] == 0.5:
            continue
        
        total_count += 1
        if insights[specie][0] == insights[specie][1][0]:
            correct_count += 1
            correct_species += 1
        
        if insights[specie][1][0] == insights[specie][0] == 1:
            true_positives += 1
        elif insights[specie][1][0] == insights[specie][0] == -1:
            true_negatives += 1
        elif insights[specie][1][0] == 1 and insights[specie][0] == -1:
            false_positives += 1
        elif insights[specie][1][0] == -1 and insights[specie][0] == 1:
            false_negatives += 1
        
        
         
    print(directory, ' total species: ', total_count, 'correct species: ', correct_count, 'ratio: ', correct_count/total_count)
    complete_results[directory] = correct_count / total_count
    

Converged at time 6792.0
Converged at time 7974.0
Specie:  M expected result:  decreased  normal final:  323.7613772367963  altered final:  8.148296173275313
confidence interval:  [0.5318330143063276, 0.5439597669586405]
deltas coming from simulation:  [-315.613081063521, 0]
Specie:  Mp expected result:  decreased  normal final:  9.496206386452487  altered final:  4.8464454526800855
confidence interval:  [0.09906314070255157, 0.10815952911213307]
deltas coming from simulation:  [-4.649760933772401, 0]
Specie:  Mpp expected result:  increased  normal final:  37.10128283610644  altered final:  383.9666704398843
confidence interval:  [0.6020972683611978, 0.6221446744135689]
deltas coming from simulation:  [346.86538760377783, 0]
Specie:  MAPKK expected result:  increased  normal final:  6.729376566114122  altered final:  42.696896875695984
confidence interval:  [0.5757157537298163, 0.5849181624862136]
deltas coming from simulation:  [35.967520309581865, 0]
Specie:  MKP3 expected result:  

In [3]:
scores = []
for experiment in complete_results.keys():
    scores.append(complete_results[experiment])
print('Average score: ', np.average(np.array(scores)))

Average score:  0.6931818181818181


In [4]:
results_path = '../../../results/real_dataset/large_class/results'
results_file = open(results_path, 'rb')
partial_results = pickle.load(results_file)
results_file.close()
partial_results

{'total_species': 442,
 'uncertain_species': 22,
 'correct_species': 313,
 'true_positives': 159,
 'false_positives': 62,
 'false_negatives': 67,
 'true_negatives': 154}

In [4]:
print('total: ', total_species, ' correct: ', correct_species, ' uncertain: ', uncertain_species, ' true pos: ', true_positives, ' true neg: ', true_negatives, ' false_pos: ', false_positives, 'false neg: ', false_negatives,)

total:  176  correct:  122  uncertain:  0  true pos:  57  true neg:  65  false_pos:  31 false neg:  23


In [6]:
partial_results['total_species'] += total_species
partial_results['uncertain_species'] += uncertain_species
partial_results['correct_species'] += correct_species
partial_results['true_positives'] += true_positives
partial_results['false_positives'] += false_positives
partial_results['true_negatives'] += true_negatives
partial_results['false_negatives'] += false_negatives

In [7]:
results_path = '../../../results/real_dataset/large_class/results'
results_file = open(results_path, 'wb')
pickle.dump(partial_results, results_file)
results_file.close()

In [4]:
scores

[0.5454545454545454,
 0.5454545454545454,
 0.45454545454545453,
 0.7272727272727273,
 0.7272727272727273,
 0.8181818181818182,
 0.5454545454545454,
 0.8181818181818182,
 0.6363636363636364,
 0.9090909090909091,
 0.8181818181818182,
 0.6363636363636364,
 0.8181818181818182,
 0.5454545454545454,
 0.9090909090909091,
 0.7272727272727273]

In [ ]:
description

In [ ]:
description['altered'] = 'v1a'
description

In [ ]:
graph = functions.generate_graph(description)
functions.plot_graph(graph)

In [ ]:
description_path = 'description_reaction1_altered'
description_file = open(description_path, 'wb')
pickle.dump(description, description_file)
description_file.close()

In [ ]:
final_concentrations_normal = functions.plot_simulation_complete('BIOMD0000000026.xml')

In [ ]:
final_concentrations_altered = functions.plot_simulation_complete('BIOMD0000000026_reaction1_100x.xml')

In [ ]:
species = description['species']
species

In [ ]:
final_normal = final_concentrations_normal[0][-1]
final_altered = final_concentrations_altered[0][-1]

In [ ]:
final_normal

In [ ]:
temp = final_concentrations_normal[1]
temp

In [ ]:
import math
expected_results = {}
for i in range(len(species)):
    delta1 = 0
    delta2 = 0
    print(species[i])
    if final_normal[i + 1] != 0:
        if temp[i] != 0:
            initial_concentration = temp[i]
            delta1 = math.fabs((final_altered[i + 1] - final_normal[i + 1]) / initial_concentration)
            delta2 = math.fabs(1 - (final_altered[i + 1] / final_normal[i + 1]))
            paired_delta = delta1 * delta2
        else:
            delta2 = math.fabs(1 - (final_altered[i + 1] / final_normal[i + 1]))
            paired_delta = delta2 / 100
            
        relative_delta = (final_altered[i + 1] - final_normal[i + 1]) / final_normal[i + 1]
        
    else: 
        relative_delta = 0
        
    if paired_delta >= 0.001 and final_altered[i + 1] < final_normal[i + 1]:
        expected_results[species[i]] = 'decreased'
    elif paired_delta >= 0.001 and final_altered[i + 1] > final_normal[i + 1]:
        expected_results[species[i]] = 'increased'
    else:
        expected_results[species[i]] = 'unchanged'
    
    print('final normal: ', final_normal[i + 1], 'final altered: ', final_altered[i + 1], 'delta: ', final_altered[i + 1] - final_normal[i + 1], 'relative delta ', relative_delta, 'initial concentration: ', temp[i], 'delta1', delta1, 'delta2', delta2)

In [ ]:
expected_results

In [ ]:
description['reactions'][13][1] = 0.2

In [ ]:
for i in range(len(description['reactions'])):
    description['reactions'][i][2] = random.uniform(0, 1)
    description['reactions'][i].append([])

In [ ]:
graph = graph_functions.generate_graph(description)

In [ ]:
species_propagation = []
for i in range(100):
    print("ITERATION ", i)
    species_propagation.append(graph_functions.update_graph(graph, 1, i + 1))

In [ ]:
species_propagation

In [ ]:
import matplotlib.pyplot as plt
concentrations = [conc['MAPKK'] for conc in species_propagation]
concentrations
plt.plot(concentrations[0:500])

In [ ]:
final_propagation = species_propagation[-1]
for s in final_propagation.keys():
    print(s, 'propagation results: ', final_propagation[s], 'simulation results: ', expected_results[s])

In [ ]:
res = complete_tests.complete_tests2(
            'BIOMD0000000026.xml',
            'altered_pathways/reaction_12/reaction_12100x.xml',
            'altered_pathways/reaction_12/description',
            []
        )

In [ ]:
insights = complete_tests.get_insigths('altered', res, 1, detailed_classification=True, threshold=0)

In [ ]:
total_count = 0
correct_count = 0
for specie in insights.keys():
    total_count += 1
    if insights[specie][0] == insights[specie][1]:
        correct_count += 1
print('total experiments: ', total_count, 'correct experiments: ', correct_count, 'ratio: ', correct_count/total_count)

In [ ]:
total_count = 0
correct_count = 0
for specie in insights[2].keys():
    if expected_results[specie] != 'unchanged':
        total_count += 1
    if type(specie) != type(''):
        continue
    print('specie: ', specie)
    scores = np.array(insights[2][specie][-1])
    mean = np.mean(scores)
    std = np.std(scores)
    std_error = std / np.sqrt(len(scores))
    margin = 2 * std_error
    lower = mean - margin
    higher = mean + margin
    print(' lower: ', lower, ' higher: ', higher)
    if lower > 0.5: 
        print('increase, expected: ', expected_results[specie])
        if expected_results[specie] == 'increased':
            correct_count += 1
        
    else: 
        print('decrease, expected: ', expected_results[specie])
        if expected_results[specie] == 'decreased':
            correct_count += 1
print('ratio correct: ', correct_count / total_count, 'correct: ', correct_count, 'total: ', total_count)

In [ ]:
import pandas as pd 
import seaborn as sns
scores = np.array(insights['MAPKK'][-1])
df = pd.DataFrame(scores)
sns.histplot(df, kde=True, bins=100)

In [ ]:
insights['MAPKK']